In [ ]:
import pandas as pd
import geopandas as gp
import sqlalchemy as sql
import pymssql
import numpy as np

In [ ]:
def Demographics_Geographis_join (MGRAgeo, demographics, special_district_path):
    
    # Read in the MGRA Shape File
    # Get the MGRA centroids from the B8 database
    B8 = sql.create_engine('mssql+pymssql://xxxxxx/')
    weighted_centroid = pd.read_sql(MGRAgeo, B8.connect())
    #display(weighted_centroid)

    # Demographic Data from SQL Server
    # Get the demographic data from the Ddam database
    ddam = sql.create_engine('mssql+pymssql://xxxxxx/')
    demographic_data = pd.read_sql(demographics,con=ddam.connect())

    # Get the Shape to the demographic data imported from the database
    Joined_table = pd.merge(weighted_centroid, demographic_data, left_on = "MGRA", right_on = "mgra", how = "inner")
    # Turn the table to geometry table and set the crs (Coordinate Reference Systems)
    Joined_table['Point'] = gp.GeoSeries.from_wkt(Joined_table['Point'])
    Joined_table = gp.GeoDataFrame(Joined_table, crs="EPSG:2230", geometry='Point')
    
    # Sjoin to determine the mgra weighted centroids that are in the Special districts.
    Special_District = gp.read_file(special_district_path)
    # Set the coordinate system 
    Special_District = Special_District.to_crs('EPSG:2230')
    joined_Geo = gp.sjoin(Joined_table, Special_District, how='right', op='within')

    return joined_Geo

In [ ]:
MGRAgeo = "SELECT [MGRA], [Shape].ToString() AS Point FROM [GeoDepot].[gis].[MGRA13_WEIGHTEDCENTROID]"

demographics = """   
SELECT d.mgra, g.income_group_id ,g.name AS income, SUM(i.households) AS households
FROM demographic_warehouse.fact.household_income i 
JOIN demographic_warehouse.dim.income_group g ON i.income_group_id = g.income_group_id
JOIN demographic_warehouse.dim.mgra_denormalize d ON i.mgra_id = d.mgra_id
WHERE datasource_id = 45 AND yr_id = 2021
GROUP BY d.mgra, g.name, g.income_group_id
ORDER BY d.mgra, g.income_group_id
"""

special_district_path = "SW_Dir_Div/SW_Dir_Divisions.shp"


Demographics_geometry = Demographics_Geographis_join(MGRAgeo,demographics,special_district_path)